In [ ]:
!pip uninstall mediapipe -y
!pip install --force-reinstall mediapipe

In [ ]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

In [ ]:
from google.colab import drive
import os

# Mount Google Drive to access your file
drive.mount('/content/drive')

# Now check where your file actually is
print("🔍 Searching for pose_landmarker_heavy.task in Drive...")

# Search everywhere in Drive
!find /content/drive -name "pose_landmarker_heavy.task" 2>/dev/null

# Common locations - try these:
possible_paths = [
    '/content/drive/MyDrive/pose_landmarker_heavy.task',
    '/content/drive/My Drive/pose_landmarker_heavy.task',  # Note the space
    '/content/drive/MyDrive/Colab Notebooks/pose_landmarker_heavy.task',
    '/content/drive/MyDrive/models/pose_landmarker_heavy.task'
]

for path in possible_paths:
    if os.path.exists(path):
        print(f"✅ Found at: {path}")
        model_path = path
        break
else:
    print("❌ File not found in common locations")
    # Let's see what's in the main Drive folder
    print("\n📁 Contents of /content/drive/MyDrive:")
    !ls -la "/content/drive/MyDrive"

In [ ]:
from google.colab import files
uploaded = files.upload()
video_filename = list(uploaded.keys())[0]

In [ ]:
import cv2
import mediapipe as mp
from google.colab.patches import cv2_imshow


def blur_faces(img, blur_strength=(55, 55), confidence=0.5):
    """
    Blurs all detected faces in an image (numpy array) using Mediapipe + OpenCV.

    Parameters:
        img (np.array): Input image (BGR format).
        blur_strength (tuple): Gaussian blur kernel size (must be odd numbers).
        confidence (float): Minimum confidence for face detection.

    Returns:
        np.array: Image with blurred faces.
    """
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection(
        model_selection=0,
        min_detection_confidence=confidence
    )

    image = img.copy()
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image_rgb)

    if results.detections:
        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = image.shape
            x, y = int(bbox.xmin * w), int(bbox.ymin * h)
            w_box, h_box = int(bbox.width * w), int(bbox.height * h)

            # Clamp coordinates to image boundaries
            x, y = max(0, x), max(0, y)
            w_box, h_box = min(w_box, w - x), min(h_box, h - y)

            # Apply blur to the detected face region
            roi = image[y:y+h_box, x:x+w_box]
            if roi.size > 0:
                blur = cv2.GaussianBlur(roi, blur_strength, 30)
                image[y:y+h_box, x:x+w_box] = blur

    face_detection.close()
    return image

In [ ]:
import cv2
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from google.colab.patches import cv2_imshow
import IPython.display as display
import time

def process_image(img):
    base_options = python.BaseOptions(model_asset_path=model_path)
    options = vision.PoseLandmarkerOptions(
        base_options=base_options,
        output_segmentation_masks=True)
    detector = vision.PoseLandmarker.create_from_options(options)
    img=blur_faces(img)
    rgb_frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    detection_result = detector.detect(image)
    annotated_image = draw_landmarks_on_image(image.numpy_view(), detection_result)

        # 🔥 CRITICAL: Clear previous output and show current frame
    display.clear_output(wait=True)

    display_frame = cv2.cvtColor(annotated_image, cv2.COLOR_RGB2BGR)
    cv2_imshow(display_frame)
    return display_frame


In [ ]:
import cv2
cap = cv2.VideoCapture(video_filename)


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("✅ Video processing completed!")
        break
    out = process_image(frame)
    blur_faces(out)
